In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import pandas as pd
import numpy as np
import h5py 
import pickle
import os
from tqdm import tqdm

In [4]:
!pwd
work_dir = '~/text_sum/'
data_folder = 'preprocessed/bert_data_cnndm_final/'


aa = torch.load(data_folder + 'cnndm.test.0.bert.pt')

/home/bui.thuy/text_sum


In [5]:
# print(aa)
print(type(aa))
print(aa[0].keys())
print(aa[0]['src'])

<class 'list'>
dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
[101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 3190, 2181, 1010, 2485, 2000, 2010, 2155, 2188, 1999, 8904, 3449, 9644, 1012, 102, 101, 2002, 2351, 2006, 4465, 2012, 7855, 3986, 2902, 1011, 2966, 19684, 1005, 2015, 2436

In [6]:
import glob

# get all train pt files
train_data_files = glob.glob(data_folder + '*train*.pt')
test_data_files = glob.glob(data_folder + '*test*.pt')
valid_data_files = glob.glob(data_folder + '*valid*.pt')

print(test_data_files)
# merge all pt file into 1 list

['preprocessed/bert_data_cnndm_final/cnndm.test.0.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.5.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.1.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.3.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.2.bert.pt', 'preprocessed/bert_data_cnndm_final/cnndm.test.4.bert.pt']


In [17]:
max_words_per_sample = 512

def merge_pt(pt_files, field, max_words_per_sample = 512):
  out_arr = None # numpy array

  # max_words_per_sample = 0
  n_samples = 0
  for fn in pt_files:
    # print('processing ', fn)
    temp_pt = torch.load(fn)
    for pt in temp_pt:
      # max_words_per_sample = max(max_words_per_sample, len(pt[field]))
      n_samples += 1

  out_arr = -np.ones(shape = (n_samples, max_words_per_sample))
  i = 0
  for fn in pt_files:
    temp_pt = torch.load(fn)
    for pt in temp_pt:
        if (len(pt[field]) < max_words_per_sample):
            out_arr[i, 0:len(pt[field])] = pt[field]
        else:
            out_arr[i, :] = pt[field][:max_words_per_sample]    
    i+=1
  return out_arr


def merge_text_pt(pt_files, field):
  # field = 'src_txt' or 'tgt_txt'
  
  out_list = []
  for fn in pt_files:
    print('processing ', fn)
    temp_pt = torch.load(fn)
    for pt in temp_pt:
      out_list.append(pt[field])
  return out_list

# merged_src_test = merge_pt(test_data_files, 'src')
# print(merged_src_test)
  


In [15]:
!pwd
!ls

/home/bui.thuy/text_sum
bert2bert.ipynb  encoder_decoder2.ipynb  preprocess.ipynb  train2.ipynb
bert_test.py	 preprocessed		 README.md	   train.ipynb


In [20]:
# merge all numeric fields in pt files
keys = list(aa[0].keys())[:-2] # ['src', 'tgt', 'src_sent_labels', 'segs', 'clss']

# print('loading test_data')
# with h5py.File("./preprocessed/test_data.hdf5", "w") as data_file:
#   for k in keys:
#     print('processing key ', k)
#     merged_test = merge_pt(test_data_files, k)
#     data_file.create_dataset(name = k, data=merged_test)
# data_file.close()
# print('done for test.hdf5')


print('loading valid_data')
with h5py.File("./preprocessed/valid_data.hdf5", "w") as data_file:
  for k in keys:
    print('processing key ', k)
    merged_valid = merge_pt(valid_data_files, k)
    data_file.create_dataset(name = k, data=merged_valid)
data_file.close()
print('done for valid.hdf5')

# print('processing train_data')
# with h5py.File("./preprocessed/train_data.hdf5", "w") as data_file:
#   for k in keys:
#     print('processing key ', k)
#     merged_train = merge_pt(train_data_files, k)
#     data_file.create_dataset(name = k, data=merged_train)
# data_file.close()

loading valid_data
processing key  src
processing key  tgt
processing key  src_sent_labels
processing key  segs
processing key  clss
done for valid.hdf5


In [19]:
data_dir = "./preprocessed/"
train_path = data_dir + 'train_data.hdf5'
test_path = data_dir + 'test_data.hdf5'
valid_path = data_dir + 'valid_data.hdf5'

f = h5py.File(test_path, 'r')
for key in f.keys():
    print(key)
    
f = h5py.File(train_path, 'r')
for key in f.keys():
    print(key)
    
f = h5py.File(valid_path, 'r')
for key in f.keys():
    print(key)

test_src = f['src']
print(test_src[:6,:])
print(test_src.shape)


clss
segs
src
src_sent_labels
tgt
[[  101.  2002.  2001. ...  2665.  2076.  1037.]
 [  101.  1006. 13229. ...  2088.  2064.  4025.]
 [  101.  2019. 10812. ...  1005.   102.   101.]
 [  101.  1037.  1036. ...  2361.  1024.  2019.]
 [  101.  5293.  1996. ...  1036.  2057.  4669.]
 [  101.  4754. 20098. ...  4239.  2223.  3966.]]
(11489, 512)


In [20]:
import sys
sys.getsizeof(merged_train)

merged_train = merge_pt(train_data_files, 'src')
print('merged_train size in mb', merged_train.nbytes / 1000000)

del merged_train

merged_train size in mb 8798.519784


In [ ]:
train_data_file = './cnn_dailymail/train.csv'
train_data = pd.read_csv(train_data_file)
vocab = {}

for idx in train_data.shape[0]:
    article = train_data.iloc[idx]['article']
    summary = train_data.iloc[idx]['highlights']
    


In [3]:
from transformers import BertTokenizer, BertModel

In [4]:
# try 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
print(aa[0].keys())

dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])


In [39]:
sents = aa[0]['src_txt']
token_ids = aa[0]['src']

print(len(token_ids))
# print(token_ids[0])
print(sents[0])
print(sents[1])

unflat = [tokenizer.encode(s) for s in sents]
flat_list = [item for sublist in unflat for item in sublist]
print(flat_list)
print(len(flat_list))

print(token_ids)
print(len(token_ids))

for i in range(len(flat_list)):
    if flat_list[i] != token_ids[i]:
        print(i, token_ids[i], flat_list[i] )
        print(tokenizer.decode(token_ids[i]))
        print(tokenizer.decode(flat_list[i]))

479
a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .
andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .
[101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 319

In [23]:
print(aa[0].keys())
for k in aa[0].keys():
    print(k, len(aa[0][k]) )
    print(aa[0][k])

dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
src 479
[101, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 2886, 1999, 4199, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 2018, 2069, 2074, 3369, 2005, 1037, 13609, 2565, 1999, 3304, 2043, 1996, 5043, 3047, 1999, 2254, 1012, 102, 101, 2002, 2001, 10583, 2067, 2000, 3190, 3081, 2250, 10771, 2006, 2233, 2322, 1010, 2021, 2002, 2351, 2006, 4465, 1012, 102, 101, 4080, 9587, 29076, 1010, 2322, 1010, 2013, 8904, 3449, 9644, 1010, 4307, 1010, 1037, 2118, 1997, 5947, 3076, 2038, 2351, 3053, 2093, 2706, 2044, 1037, 2991, 1999, 4199, 1999, 1037, 6878, 13742, 102, 101, 2002, 2001, 2579, 2000, 1037, 2966, 4322, 1999, 1996, 3190, 2181, 1010, 2485, 2000, 2010, 2155, 2188, 1999, 8904, 3449, 9644, 1012, 102, 101, 2002, 2351, 2006, 4465, 2012, 7855, 3986, 2902, 1011, 2966, 19684, 1005, 2015, 2436, 14056